Herramientas utilizadas: Pandas y matplotlib

Gráfico Ciruclar de vuelos filtrados por la fecha de vuelo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Leo el dataframe
df = pd.read_csv('itineraries_clean.csv')
df = df[['legId', 'startingAirport', 'destinationAirport']]

# Me quedo con los vuelos únicos, evito repeticiones del mismo vuelo
df = df.drop_duplicates(subset=['legId'])

# Convertir a objeto datetime para poder filtrar bien
df['flightDate'] = pd.to_datetime(df['flightDate'])
start_date = pd.Timestamp('2022-07-01')  # Fecha de inicio
end_date = pd.Timestamp('2022-07-31')  # Fecha de fin
df = df.loc[((df['flightDate'] >= start_date) & (df['flightDate'] <= end_date)) & (df['startingAirport'] == 'ATL')]

df = df[['startingAirport', 'destinationAirport']]
df = df.groupby(['startingAirport', 'destinationAirport']).size().reset_index(name='Count')

# Generar gráfico circular
plt.pie(df['Count'], labels=df['startingAirport'] + ' to ' + df['destinationAirport'], autopct='%1.1f%%')
plt.title('Apariciones de cada par de aeropuertos de origen y destino (Julio 2022)')

plt.savefig('grafico_aeropuertos.png', bbox_inches='tight')

Número de vuelos por mes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Leo el dataframe
df = pd.read_csv('itineraries_clean.csv')
df = df[['legId', 'startingAirport', 'destinationAirport', 'flightDate']]

# Convertir a objeto datetime para poder sacar el mes del vuelo
df['flightDate'] = pd.to_datetime(df['flightDate'])
df['month'] = df['flightDate'].dt.month

# Consigo una serie que contiene el mes y el número de vuelos
flight_count = df['month'].value_counts().sort_index()

# Creo un gráfico de barras
fig, ax = plt.subplots()
ax.bar(flight_count.index, flight_count.values)

# Genero un gráfico de barras
ax.set_xlabel('Mes')
ax.set_ylabel('Número de vuelos')
ax.set_xticks(range(1, 8))
ax.set_xticklabels(['Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov'])

plt.savefig('vuelos_por_mes.png')

Herramientas utilizadas: Spark

In [ ]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt

# Crear una sesión de Spark
spark = SparkSession.builder.appName('numAerolineas').getOrCreate()

# Leer el archivo csv y crear el DataFrame
df = spark.read.option("header", True).csv("itineraries_clean.csv")

# Seleccionar la columna de aerolíneas y separarlas por "||" si hay varias en una misma fila
rdd = df.rdd.flatMap(lambda x: x['segmentsAirlineName'].split('||'))

# Contar las apariciones de cada aerolínea
rdd = rdd.map(lambda x: (x.strip(), 1)).reduceByKey(lambda a, b: a + b)

# Crear el DataFrame de pandas y ordenarlo de mayor a menor
df_pandas = rdd.toDF(['aerolinea', 'count']).sort('count', ascending=False).toPandas()

# Crear el gráfico de barras horizontales
plt.barh(df_pandas['aerolinea'], df_pandas['count'])

# Configurar el título y las etiquetas de los ejes
plt.title('Conteo de aerolíneas')
plt.xlabel('Conteo')
plt.ylabel('Aerolínea')

plt.savefig('grafico_aerolineas.png', bbox_inches='tight')